In [1]:
# 準備工作

from bs4 import BeautifulSoup
import time
import datetime
import re
import sqlite3
import os
import json
import pprint

db_name = "../data/21_century.sqlite"
json_name = "../data/21_century.json"

In [2]:
# 從db_name取出資料

with sqlite3.connect(db_name) as conn_retrieve:
    
    c_rt = conn_retrieve.cursor()
    qryString = "SELECT * from rental;"
    c_rt.execute(qryString)
    retrieved_list = c_rt.fetchall()

c_rt.close()

# 定義方法

In [3]:
# 定義 recover_quot() 方法 (recover quotation marks)
# 將字串中連續兩個的單引號跟雙引號復原為
# 單一的單引號跟雙引號

def recover_quot(string):
    string = string.replace("\"\"","\"")
    string = string.replace("\'\'","\'")
    return string

In [4]:
# 定義 remove_dust() 方法
# 清理一個字串中所有的 \r, \n, \t, \xa0 符號
# 並將頭尾的空格去掉

def remove_dust(string):
    string = string.replace("\r","")
    string = string.replace("\n","")
    string = string.replace("\t","")
    string = string.replace("\xa0","")
    string = string.strip()
    return string

In [5]:
# 定義 remove_dirt() 方法
# 用於移除'環境介紹'中的雜物

def remove_dirt(text):
    
    replace_set = [('：',':'),('︰',':'),(', ',','),
                  ('、 ','、'),('\xa0',''),
                  ('ˊ',''),('～',''),('※',''),
                  ('查詢主頁動態',''),('特色說明','\r特色說明:'),
                  ('租金不含這些','租金不含'),('租金不含','\r租金不含:'),
                  ('備註:歡迎房客加入line或微信 ID','\r網路社群ID'),
                  ('歡迎房客加入line或微信 ID','\r網路社群ID'),
                  ('位置最大特色','\r位置最大特色'),
                  ('押金為','押金:'),('個月','個月\r解說:,'),
                  (' ','')]

    for pair in replace_set:
        text = re.sub(pair[0],pair[1],text).strip()
        
    return text

In [6]:
# 定義 remove_multi_space() 方法
# 清理多空格

def remove_multi_space(string):
    while "  " in string:
        string = string.replace("  ","")
    return string

In [7]:
# 定義 tail_pos() 方法
# 從字串中找出子字串的所有的尾端位置
# 如果找不到就傳回空的list

def tail_pos(substring, string):
    length = len(substring)
    return [pos.start() + length for pos in 
            re.finditer(substring,string)]

In [8]:
# 定義 is_col_before_break() 方法
# 判斷一個字串中分號是否在換行符號前面
# 或不存在

def is_col_before_break(string):
    try:
        first_colon_pos = string.index(':')
        try:
            first_break_pos = string.index('\n')
            return first_break_pos > first_colon_pos
        except:
            return True 
    except:
        return False

In [9]:
# 定義 desc_as_dict 方法
# 將'環境介紹'的內容拆成dictionary

def desc_as_dict(desc):

    desc_dict = {}
    
    if type(desc) != None:
                   
        if len(desc) > 0:

            # 從tag中取出文字並存放在list中
            
            list_1 = [each.text+"\r" for each in desc.select('p')]

            
            # 將各項目細分後裝到新的list中

            list_2 = []

            for each in list_1:
                each = remove_dirt(each)                # 清理
                each = re.split('\n|\r|●|◆',str(each))  # 細分內文
                for each2 in each:
                    black_list = ['',' ','\xa0','●','◆']# 排除異樣字元
                    if each2 not in black_list:
                        list_2.append(each2)            # 存入list_2

            # 將沒有引號的句子跟前一句合併

            # 先檢查前面頭幾句
            # 前面頭幾句比較可能為獨立項目 而不是同一個
            
            list_len_2 = len(list_2)
            
            if list_len_2 > 0:

                i = 0
                
                for i in range(0,list_len_2):
                    if ':' not in list_2[i]:
                        list_2[i] = "項目" + str(i) + ":" + list_2[i]
                        i += 1

            # 找關鍵字 如果找到的話就加在前面作為key值
            
            keyword_list = [('養寵物','養寵物'),
                           ('坪','坪數'),
                           ('交屋','交屋'),
                           ('開伙','開伙'),
                           ('短租','短租'),
                           ('設籍或登記','設籍或登記')]
            
            lenght_2 = len(list_2)
            
            for k in range(0,lenght_2):
                for keyword in keyword_list:
                    if ":" not in list_2[k] and keyword[0] in list_2[k]:
                        list_2[k] = keyword[1] + ":" + list_2[k]
                    
            # 接下來合併後面的句子

            list_3 = []

            while len(list_2) > 0:
                if ":" in list_2[0]:
                    list_3.append(list_2.pop(0))
                else:
                    list_3[-1] = list_3[-1] + list_2.pop(0)


            # 轉為dictionary

            for each in list_3:
                pair = each.split(":")
                desc_dict.update({pair[0]:pair[1]})

    # 傳回值

    return desc_dict

In [10]:
# 定義 get_section_data() 方法
# 從每個章節中抽出資料

def get_section_data(soup):
    
    section_data = {}
    sections = [each.text for each in soup.find_all('h3')]
    
    for section_name in sections:

        section = soup.find_all('h3',text=section_name)[0]
        section = section.find_parent()

        if not section_name == "環境介紹":

            section_key_tags = section.select('h6')
            section_keys = [each.text for each in section_key_tags]
            section_values = [each.find_next().text for each in section_key_tags]

            # 清理不必要的字串
        
            section_keys = [remove_dust(each) for each in section_keys]
            section_values = [remove_dust(each) for each in section_values]
            
        else:

            section_keys = ['環境介紹']
            desc = section.select_one('div > div')
            section_values = [desc_as_dict(desc)]
            
            if len(section_values) == 0:
                section_values = [""]
    
        
        # 塞值到 section_dict 中
        section_data.update(dict(zip(section_keys, section_values))) 
    
    # 回傳
    return section_data

In [11]:
# 定義 get_agent_data() 方法
# 從soup中取得仲介資訊

def get_agent_data(suop):

    try:
 
        agent_tags = soup.find_all('div',class_=re.compile("column staff"))
        agent_names = [each.select_one('h4').text for each in agent_tags]

        # 證號

        agent_licenses = [remove_dust(each.find('div',class_="LicenseNo").text) for each in agent_tags]

        # 電話

        agent_contact_tags = soup.select('div[name="agentContactInfo"]')
        agent_phones = []

        # 先找出每個仲介的所有電話(可能不只1個) 個別存成list
        # 但因為select後每個仍為含有tag的list
        # 所以得再將每個list裡面的tag挑出來 取出真正的電話資料後再存回

        for each_list in agent_contact_tags:
            each_agent_phones = []
            each_list = each_list.select('span[class="mobileInfo"] > a')

            for each in each_list:
                each_agent_phones.append(each.text)

            agent_phones.append(each_agent_phones)


        # email
        # 跟類似處理電話的方式處理

        agent_emails_tags = [each.find_all('a',href=re.compile("mailto:")) for each in agent_contact_tags]
        agent_emails = []

        for each_list in agent_emails_tags:
            each_agent_emails = []  

            for each in each_list:
                each_agent_emails.append(each.text)

            agent_emails.append(each_agent_emails)

        # 整合仲介資料

        agent_data = {}

        for j in range(0,len(agent_names)):

            agent_data.update(
                {agent_names[j]: {    # 代表仲介姓名
                    '證號': agent_licenses[j],
                    '電話': agent_phones[j],
                    'email': agent_emails[j]  
                }
            })

    except:
        agent_data = "NULL"
        
    finally:
        return agent_data

In [12]:
# 定義 get_landlord_data() 方法
# 從soup中取得房東資訊

def get_landlord_data(soup):

    try:        
        section = soup.find('div',class_=re.compile("column owner"))
        landlord_name = section.select_one('h3').text
        landlord_details = section.select_one('span').text

        landlord_data = {
            'name': landlord_name,
            'details': landlord_details
        }
    
    except:
        landlord_data = "NULL"
    
    finally:
        return landlord_data

In [13]:
# 定義 get_branch_data() 方法
# 從soup中取得加盟店資訊

def get_branch_data(soup):
    
    try:
        branch_tags = soup.find('div',class_=re.compile("column store"))
        branch_name = branch_tags.select_one('h4').text
        branch_co_name = branch_tags.select('ul > li')[0].text
        branch_address = branch_tags.select('ul > li')[1].text
        branch_phone = branch_tags.select('ul > li')[2].text
        branch_fax = branch_tags.select('ul > li')[3].text

        branch_data = {
            '名稱': branch_address,
            '分行公司': branch_co_name,  
            '地址': branch_address,
            '電話': branch_phone,
            '傳真': branch_fax
        }

        # 整理加盟店資訊

        branch_data = {key:remove_dust(branch_dict[key]).strip() for key in store_dict }
        
    except:
        branch_data = "NULL"
        
    finally:
        return branch_data

In [14]:
# 建立 get_data_dict 方法
# 給代表網頁內容的soup物件
# 取得每頁的資料 並以dictionary形式傳回

def get_data_dict(soup):

    # 內頁標籤1(如"租屋")
#     print(soup.select('div'))
    tag01 = soup.select('div[id="breadCrumb"] > ul > li')[1].text

    # 內頁標籤2(如"租屋列表")
    tag02 = soup.select('div[id="breadCrumb"] > ul > li')[2].text

    # 內頁標籤3(如"光復南路優質地段B1(租10)")
    tag03 = soup.select('div[id="breadCrumb"] > ul > li')[3].text

    # 經緯度
    
    try:
        lat_lng_source = soup.find(text=re.compile('LatLng\(\d'))
        lat_lng = re.search('(?<=LatLng\().*?(?=\))',lat_lng_source).group()
        lat_lng = re.split('(?<=\d)[^\d]*?,.*?(?=\d)',lat_lng)
        lat = float(lat_lng[0])
        lng = float(lat_lng[1])
    except:
        lat = 0
        lng = 0
    
    # 仲介資訊    
    agent_data = get_agent_data(soup)

    # 房東資訊
    landlord_data = get_landlord_data(soup)

    # 加盟店
    branch_data = get_branch_data(soup)
    
    # 存放單筆資料於dictionary中
    data_dict = {
        '標籤01': tag01,
        '標籤02': tag02,
        '地點名稱': tag03,
        '經度': lng,
        '緯度': lat,
        '仲介': agent_data,
        '房東': landlord_data,
        '分行': branch_data
    }

    # 跟其他dictionary合併
    data_dict.update(get_section_data(soup))

    # 傳回結果
    return data_dict

# 執行程序

In [15]:
# 跑迴圈 清洗資料
# 並將結果暫存到一個大的dictionary中

# index_set = range(0,10)    # 測試用
index_set = range(0,len(retrieved_list))
dict_all = {}

for i in index_set:

#     print(retrieved_list[i][0])
    
    # 將內頁內容轉成 BeautifulSoup 物件

    page_source = retrieved_list[i][1]
    page_source = recover_quot(page_source) # 還原引號
    soup = BeautifulSoup(page_source,'html.parser')

    # 網址
    # posting_id 為上架ID

    url = retrieved_list[i][0]
    posting_id = url.split("/")[-1]
   
    # 清理

    data_dict = get_data_dict(soup)
    data_dict.update({'url':url})
    data_dict.update({'爬蟲日期時間': retrieved_list[i][4]})

    # 暫存

    dict_all.update({posting_id: data_dict})

In [16]:
# 將最後的結果 dict_all 以json格式存到 json_name 路徑裡

if os.path.isfile(json_name) == False:
    json = json.dumps(dict_all,ensure_ascii=False)
    f = open(json_name,"w")
    f.write(json)
    f.close()

In [18]:
# pprint.pprint(dict_all)

{'10669': {'url': 'http://www.century21.com.tw/index/Rental/RentPage/10669',
           '仲介': {},
           '其他說明': '可養寵物、不與房東同住',
           '分行': 'NULL',
           '地址': '台北市中山區建國北路三段',
           '地點名稱': '濱江陽光雅寓住家~2大套房',
           '坪數': '32坪',
           '性別限制': '男女皆可',
           '房東': {'details': '方便聯絡時間：09:00~12:00 或 14:00~22:00', 'name': '藍'},
           '押金': '2個月',
           '樓層': '5/5樓',
           '標籤01': '租屋',
           '標籤02': '租屋列表',
           '爬蟲日期時間': '2017-06-07 22:06:22',
           '環境介紹': {},
           '租金': '每月30,000元',
           '種類': '公寓',
           '經度': 121.5339826,
           '緯度': 25.0680141},
 '11876': {'url': 'http://www.century21.com.tw/index/Rental/RentPage/11876',
           '仲介': {'陳秀吟': {'email': ['show483055@yahoo.com.tw'],
                          '證號': '營業員證號：(104)登字第295492號',
                          '電話': ['0933075795']}},
           '其他說明': '不與房東同住',
           '分行': 'NULL',
           '地址': '新北市三芝區田心子',
           '地點名稱': '田心子廠房出租',
 

           '租金': '每月32,000元',
           '種類': '大廈',
           '經度': 121.4585896,
           '緯度': 25.1308954,
           '鄰近學校': '竹圍國小、竹圍高中、竹圍國中',
           '鄰近市場': '傳統市場、全聯社、頂好超市'},
 '14464': {'url': 'http://www.century21.com.tw/index/Rental/RentPage/14464',
           '仲介': {'李大森': {'email': ['dasn5761@gmail.com'],
                          '證號': '經紀人員證號：106南市字第00598號',
                          '電話': ['0912178882', '065833311']}},
           '其他說明': '不與房東同住',
           '分行': 'NULL',
           '地址': '台南市佳里區鎮安段',
           '地點名稱': '佳里一般農業區農地',
           '坪數': '1,486.7633坪',
           '性別限制': '男女皆可',
           '房東': 'NULL',
           '押金': '面議',
           '樓層': '0樓',
           '標籤01': '租屋',
           '標籤02': '租屋列表',
           '爬蟲日期時間': '2017-06-07 22:06:37',
           '環境介紹': {},
           '租金': '每月100,000元',
           '種類': '農地',
           '經度': 0,
           '緯度': 0,
           '鄰近公園': '社區公園',
           '鄰近市場': '傳統市場'},
 '14480': {'url': 'http://www.century21.com.t

 '14794': {'url': 'http://www.century21.com.tw/index/Rental/RentPage/14794',
           '仲介': {'吳麗紅': {'email': ['ninawu0218@gmail.com'],
                          '證號': '營業員證號：(102)登字第245582號',
                          '電話': ['0923365288']}},
           '其他說明': '不與房東同住',
           '分行': 'NULL',
           '地址': '台中市西屯區市政路',
           '地點名稱': '優雅裝潢家具家電全配',
           '坪數': '63.4189坪',
           '性別限制': '男女皆可',
           '房東': 'NULL',
           '押金': '面議',
           '樓層': '0樓',
           '標籤01': '租屋',
           '標籤02': '租屋列表',
           '爬蟲日期時間': '2017-06-07 22:06:52',
           '環境介紹': {},
           '租金': '每月45,000元',
           '種類': '公寓',
           '經度': 0,
           '緯度': 0,
           '鄰近公園': '惠來公園',
           '鄰近學校': '惠文學區',
           '鄰近市場': '新光遠百'},
 '14808': {'url': 'http://www.century21.com.tw/index/Rental/RentPage/14808',
           '仲介': {'阿麗': {'email': ['changdi65@gmail.com'],
                         '證號': '營業員證號：(100)登字號175764號',
                         

           '其他說明': '不與房東同住',
           '分行': 'NULL',
           '地址': '新北市林口區文化三路一段',
           '地點名稱': '未來城店面便宜租',
           '坪數': '43.0268坪',
           '性別限制': '男女皆可',
           '房東': 'NULL',
           '押金': '6.4萬( 2個月 )',
           '樓層': '0',
           '標籤01': '租屋',
           '標籤02': '租屋列表',
           '爬蟲日期時間': '2017-06-07 22:07:08',
           '環境介紹': {},
           '租金': '每月32,000元',
           '種類': '大廈',
           '管理費': '每月2,753元',
           '經度': 121.3653103,
           '緯度': 25.0729148,
           '鄰近公園': '扶輪公園',
           '鄰近學校': '麗林國小/崇林國中/林口高中/醒吾',
           '鄰近市場': '三井OUTLET'},
 '14990': {'url': 'http://www.century21.com.tw/index/Rental/RentPage/14990',
           '仲介': {'劉美芳': {'email': ['liu651231@yahoo.com.tw'],
                          '證號': '營業員證號：102登字第240443',
                          '電話': ['0919192882']}},
           '其他說明': '不與房東同住',
           '分行': 'NULL',
           '地址': '高雄市前鎮區景德路',
           '地點名稱': '景德優質套房出租',
           '坪數': '7.7137坪',


           '分行': 'NULL',
           '地址': '桃園市中壢區復興路',
           '地點名稱': '復興路黃金商辦',
           '坪數': '160.2946坪',
           '性別限制': '男女皆可',
           '房東': 'NULL',
           '押金': '16萬( 2個月 )',
           '樓層': '11樓',
           '標籤01': '租屋',
           '標籤02': '租屋列表',
           '爬蟲日期時間': '2017-06-07 22:07:20',
           '環境介紹': {},
           '租金': '每月80,000元',
           '種類': '辦公',
           '管理費': '每月9,000元',
           '經度': 121.2250143,
           '緯度': 24.9576852},
 '15150': {'url': 'http://www.century21.com.tw/index/Rental/RentPage/15150',
           '交通狀況': '內湖站',
           '仲介': {'謝天龍': {'email': ['a0933027367@gmail.com'],
                          '證號': '營業員證號：(91)登字第003480號',
                          '電話': ['0933027367', '(02)6600-2121']}},
           '其他說明': '不與房東同住',
           '分行': 'NULL',
           '地址': '台北市內湖區內湖路二段',
           '地點名稱': '友座君品金店面2  租21',
           '坪數': '43.94坪',
           '性別限制': '男女皆可',
           '房東': 'NULL',
           '押金': '2個月',
   

...

           '分行': 'NULL',
           '地址': '桃園市桃園區經國路',
           '地點名稱': '(租)經國路展場三角窗店面',
           '坪數': '167.62坪',
           '性別限制': '男女皆可',
           '房東': 'NULL',
           '押金': '114萬( 3個月 )',
           '樓層': '1-2',
           '標籤01': '租屋',
           '標籤02': '租屋列表',
           '爬蟲日期時間': '2017-06-07 22:07:40',
           '環境介紹': {},
           '租金': '每月240,000元',
           '種類': '店面',
           '經度': 121.3025284,
           '緯度': 25.0229849,
           '鄰近學校': '新埔國小、經國國中',
           '鄰近市場': '同安黃昏市場'},
 '15280': {'url': 'http://www.century21.com.tw/index/Rental/RentPage/15280',
           '仲介': {'張歲沛': {'email': ['kimura.sky@msa.hinet.net'],
                          '證號': '營業員證號：(94)登字第047933',
                          '電話': ['0953755926']}},
           '其他說明': '不與房東同住',
           '分行': 'NULL',
           '地址': '台中市西區臺灣大道二段',
           '地點名稱': '台灣大道中港通商辦公',
           '坪數': '208.64坪',
           '性別限制': '男女皆可',
           '房東': 'NULL',
           '押金': '19.8萬( 2個月 )',


           '押金': '2萬( 2個月 )',
           '樓層': '2,4樓',
           '標籤01': '租屋',
           '標籤02': '租屋列表',
           '爬蟲日期時間': '2017-06-07 22:07:49',
           '環境介紹': {},
           '租金': '每月9,000元',
           '種類': '公寓',
           '經度': 0,
           '緯度': 0,
           '鄰近學校': '中正國小'},
 '15335': {'url': 'http://www.century21.com.tw/index/Rental/RentPage/15335',
           '仲介': {'陳彥雯': {'email': ['e9830111@gmail.com'],
                          '證號': '營業員證號：(99)登字第136013',
                          '電話': ['0955313145']},
                  '黃鈺婷': {'email': ['huangyuting0717@yahoo.com.tw'],
                          '證號': '營業員證號：(99)登字第146123',
                          '電話': ['0910970700']}},
           '其他說明': '不與房東同住',
           '分行': 'NULL',
           '地址': '桃園市蘆竹區南崁路一段',
           '地點名稱': '台茂店面',
           '坪數': '232.17坪',
           '性別限制': '男女皆可',
           '房東': 'NULL',
           '押金': '2個月',
           '樓層': '1-3/6樓',
           '標籤01': '租屋',
           '標籤02': '租屋列

           '鄰近公園': '江寧、榮星',
           '鄰近學校': '五常國小/中山國中',
           '鄰近市場': '濱江'},
 '15378': {'url': 'http://www.century21.com.tw/index/Rental/RentPage/15378',
           '仲介': {'王品超': {'email': ['rix3310@msn.com'],
                          '證號': '營業員證號：(105)登字第302249號',
                          '電話': ['0905730998']}},
           '其他說明': '不與房東同住',
           '分行': 'NULL',
           '地址': '台南市善化區民生路',
           '地點名稱': '善化民生路5年店住',
           '坪數': '84.1698坪',
           '性別限制': '男女皆可',
           '房東': 'NULL',
           '押金': '4.4萬( 2個月 )',
           '提供設備': '有熱水器、有瓦斯爐、有燈飾、有窗簾、有廚具',
           '樓層': '1+2樓',
           '標籤01': '租屋',
           '標籤02': '租屋列表',
           '爬蟲日期時間': '2017-06-07 22:08:01',
           '環境介紹': {},
           '租金': '每月22,000元',
           '種類': '透天店面',
           '經度': 0,
           '緯度': 0,
           '鄰近公園': '社區公園',
           '鄰近學校': '善化高中',
           '鄰近市場': '傳統市場'},
 '15381': {'url': 'http://www.century21.com.tw/index/Rental/RentPage/15381',
   

                          '電話': ['0932099858', '(03)317-9911']}},
           '其他說明': '不與房東同住',
           '分行': 'NULL',
           '地址': '桃園市桃園區新埔六街',
           '地點名稱': '(租)中悅環球企業總部',
           '坪數': '116.4坪',
           '性別限制': '男女皆可',
           '房東': 'NULL',
           '押金': '面議',
           '樓層': '20樓',
           '標籤01': '租屋',
           '標籤02': '租屋列表',
           '爬蟲日期時間': '2017-06-07 22:08:09',
           '環境介紹': {},
           '租金': '每月98,000元',
           '種類': '大廈',
           '管理費': '每月12,129元',
           '經度': 121.3022225,
           '緯度': 25.0147936,
           '鄰近公園': '中正公園',
           '鄰近學校': '慈文國小、文昌國中',
           '鄰近市場': '同安黃昏市場'},
 '15426': {'url': 'http://www.century21.com.tw/index/Rental/RentPage/15426',
           '仲介': {'江培羽': {'email': ['janiceccc1025@gmail.com'],
                          '證號': '營業員證號：(106)登字第314449號',
                          '電話': ['0929178131']}},
           '其他說明': '不與房東同住',
           '分行': 'NULL',
           '地址': '高雄市左營區榮總路',
      

           '環境介紹': {},
           '租金': '每月26,000元',
           '種類': '公寓',
           '經度': 121.5258634,
           '緯度': 25.1202274},
 '15475': {'url': 'http://www.century21.com.tw/index/Rental/RentPage/15475',
           '仲介': {'周財富': {'email': [],
                          '證號': '營業員證號：102登字第233012號',
                          '電話': ['0926117278', '(02)2737-1578']}},
           '其他說明': '不與房東同住',
           '分行': 'NULL',
           '地址': '台北市信義區松仁路',
           '地點名稱': '松仁店面(租4)',
           '坪數': '33.7898坪',
           '性別限制': '男女皆可',
           '房東': 'NULL',
           '押金': '2個月',
           '樓層': '一',
           '標籤01': '租屋',
           '標籤02': '租屋列表',
           '爬蟲日期時間': '2017-06-07 22:08:17',
           '環境介紹': {},
           '租金': '每月65,000元',
           '種類': '店面',
           '經度': 121.568077777771,
           '緯度': 25.025297113507,
           '鄰近學校': '吳興國小'},
 '15476': {'url': 'http://www.century21.com.tw/index/Rental/RentPage/15476',
           '仲介': {'廖建智': {'email': ['2